In [303]:
import sys
!{sys.executable} -m pip install tweepy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install -U textblob
!{sys.executable} -m pip install preprocessor
!{sys.executable} -m pip install tweet-preprocessor
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install argparse
!{sys.executable} -m pip install -U nltk



Requirement already up-to-date: textblob in /Users/User/anaconda3/envs/ipykernel_py3/lib/python3.7/site-packages (0.15.3)
Requirement already up-to-date: nltk in /Users/User/anaconda3/envs/ipykernel_py3/lib/python3.7/site-packages (3.4.1)
     |████████████████████████████████| 24.7MB 15.0MB/s eta 0:00:01     |█████████████████████████▊      | 19.8MB 15.7MB/s eta 0:00:01
     |████████████████████████████████| 92kB 15.0MB/s eta 0:00:01
     |████████████████████████████████| 61kB 19.0MB/s eta 0:00:01
     |████████████████████████████████| 1.4MB 39.5MB/s eta 0:00:01
     |████████████████████████████████| 133kB 30.0MB/s eta 0:00:01
     |████████████████████████████████| 71kB 6.4MB/s eta 0:00:01
     |████████████████████████████████| 5.4MB 19.0MB/s eta 0:00:01     |██████████████████████          | 3.7MB 19.0MB/s eta 0:00:01


     |████████████████████████████████| 552kB 22.7MB/s eta 0:00:01
  Stored in directory: /Users/User/Library/Caches/pip/wheels/52/5c/7d/8c9b47574380b68f7efdd637657f135ca143ea0ba4ade3b296
  Stored in directory: /Users/User/Library/Caches/pip/wheels/d5/46/82/90a89c23eac1837364ed7217a9eed71bc9e6ad4825be93968e
  Stored in directory: /Users/User/Library/Caches/pip/wheels/e4/ba/b0/1f4178a35c916b22fc51dc56f278125d4b8cfb0592e5f0cc24
  Stored in directory: /Users/User/Library/Caches/pip/wheels/b8/cb/43/c0ba52baf2b0e371ec1d5b2d4685d6d24617b1391f3eeacda5
Successfully built pycontractions language-check pyemd smart-open


In [304]:
import tweepy
import os
import sklearn
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import pandas as pd
import csv
import re #regular expression
from textblob import TextBlob
import string
import preprocessor.api as p
import argparse
import nltk
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords

In [155]:
#Twitter credentials for the app
consumer_key = 'Lydypy5GRHslhuWsXTAagVFpO'
consumer_secret = 'K9HA6MyfRWm73G50WHvzBPxfY0gWfJRk5ajcUmGRCg4e9NiM69'
access_key= '789687511-BGbhUzj8zVLk9HeKKxrCZnzJ21xb3qXqZMHyf0gX'
access_secret = 'kIDmi6vhOiePyEIZ5XXrOV8rl0xLOe5wLQ2XbhH2qCLsr'

In [156]:
#pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [157]:
#declare file paths as follows for three files
democrat_tweets = "/Users/User/221project/data/senate_democrat_data.csv"
republican_tweets = "/Users/User/221project/data/senate_republican_data.csv"

In [158]:
#columns of the csv file
COLS = ['id','clean_text', 'sentiment','polarity', 'subjectivity', 'hashtags']

In [159]:
#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

In [160]:
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

In [161]:
#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)


In [162]:
#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

In [409]:
def clean_tweets(tweet):
 
    #stop_words = set(stopwords.words('english'))
#after tweepy preprocessing the colon symbol left remain after      #removing mentions
    tweet = re.sub(r'‚Ä¶', '', tweet)
    tweet = re.sub(r'RT *[^:]*:','', tweet);
    tweet = re.sub(r':', '', tweet)
#replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
#remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
#remove url
    tweet = re.sub(r"http\S+", "", tweet)
#filter using NLTK library append it to a string
    filtered_tweet = []
    word_tokens = word_tokenize(tweet)
#looping through conditions
    for w in word_tokens:
#check tokens against stop words , emoticons and punctuations
        if w not in emoticons and w not in string.punctuation:
            if w[0] == ".":
                continue
            w = re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', w)
            w = w.lower()
            
            if len(w) == 1 and w not in ['a', 'e', 'i','o','u']:
                continue
            elif len(w) == 2 and w[0] in string.punctuation:
                continue
            filtered_tweet.append(w.lower())
    return ' '.join(filtered_tweet)
    #print(word_tokens)
    #print(filtered_sentence)return tweet

In [410]:
contractions_dict = { 
"ain't": "have not",
"aren't": "are not",
"can't": "cannot",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'll": "he will",
"he's": "he has",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "I would",
"i'll": "I will",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had",
"it'll": "it will",
"it's": "it has",
"let's": "let us",
"ma'am": "madam",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"o'clock": "of the clock",
"oughtn't": "ought not",
"she'd": "she would",
"she'll": "she will",
"she's": "she has",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there would",
"there's": "there has",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"y'all": "you all",
"you'd": "you would",
"you'll": "you will",
"you're": "you are",
"you've": "you have"
}


In [411]:
contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()), flags = re.IGNORECASE)
def expand_contractions(s, contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0).lower()]
    return contractions_re.sub(replace, s)

In [420]:
def write_tweets(handle, file):
    #If the file exists, then read the existing data from the CSV file.
    #if os.path.exists(file):
        #df = pd.read_csv(file, header=0)
   # else:
        #df = pd.DataFrame(columns=COLS)
    #page attribute in tweepy.cursor and iteration
    for page in tweepy.Cursor(api.user_timeline, id=handle,
                              count=200, include_rts=True, tweet_mode = 'extended').pages():
        for status in page:
            new_entry = []
            status = status._json
    
            #status['text'] = status['text'].replace("#", "")
            if "retweeted_status" in status:
                status['full_text'] = status["retweeted_status"]['full_text']
            
            status['full_text'] = status['full_text'].replace("’", "'")
            status['full_text'] = expand_contractions(status['full_text'])
         
            status['full_text'] = status['full_text'].replace("@", "")
            clean_text = p.clean(status['full_text'])
            filtered_tweet=clean_tweets(clean_text)

            blob = TextBlob(filtered_tweet)
            Sentiment = blob.sentiment     
            polarity = Sentiment.polarity
            subjectivity = Sentiment.subjectivity
            
            new_entry += [status['id'],filtered_tweet, Sentiment,polarity,subjectivity]
            hashtags = ", ".join([hashtag_item['text'] for hashtag_item in status['entities']['hashtags']])
            new_entry.append(hashtags) #append the hashtags
        
            with open(file, "a+", newline='') as fp:
                wr = csv.writer(fp, dialect='excel')
                wr.writerow(new_entry)
    
            #single_tweet_df = pd.DataFrame([new_entry], columns=COLS)
            #df_final = df.append(single_tweet_df, ignore_index=True)
            
            #csvFile = open(file, 'a+' ,encoding='utf-8')
            #df.to_csv(csvFile, mode='a', columns=COLS, index=False, encoding="utf-8")

In [421]:
democrat_handle = "SenateDems"
republican_handle = "SenateGOP"

write_tweets(democrat_handle, democrat_tweets)
write_tweets(republican_handle, republican_tweets)

Climate change is a public health emergency — but it has also an opportunity. Clean energy creates new jobs. And by cleaning up polluting sources we can cut emissions, AND dangerous air pollution at the same time. #ClimateHealth2019 LungAssociation #ActOnClimate
Enjoyed talking with BrownMedicine students about the future of health care. We need to improve the quality of care while expanding access in RI &amp; across the country. https://t.co/9aSbqIL8dk
This week #WeRemember the innocent victims of the Holocaust. Saturday's shooting in Poway reminds us that antisemitism and hate still exist in our communities and we must take a stand against it wherever it occurs. #HolocaustRemembrance
As we remember the millions of Jews murdered in the Holocaust this week, we have to condemn and hold accountable those who seek to perpetrate such evil and drag us toward intolerance and hatred. We can never be silent in the face of such depravity. #WeRemember https://t.co/evTQLj32cv
A very happy birthda

“While many of my Republican colleagues remain determined to turn back the clock and drag our country back to pre-Roe v. Wade days, that is just not going to happen. And as long as I am on this committee, I will not let it happen.” - SenFeinstein https://t.co/hwRViVpQkF
Attorney General Barr stopped answering questions the moment he was asked about his interactions with the Trump White House. His stonewalling and foot-dragging must stop. #ReleaseTheReport https://t.co/wGIUYoll3d
At today's HSGAC hearing, I asked how the impact of pulling aid to the Northern Triangle would affect both the fight against corruption and impunity in the region and our national security. https://t.co/gMELwf2LJl
This week, Republicans blocked our efforts to make updates to the #EqualPay Act and reaffirm that every worker in America has the right to receive equal pay for equal work through the #PaycheckFairness Act. Keep speaking out and let us make equal pay for equal work a reality. https://t.co/WDBtHBC4io
M

This is SenatorDurbin asking why more Republicans are not stepping up to the plate to address climate change. https://t.co/B7p72jPaju
Thank you to SenSchumer, SenatorTomUdall, SenJeffMerkley &amp; SenateDems for bringing the fight to pass the #ForThePeople Act to the Senate. Americans are demanding bold changes to protect our democracy &amp; end corruption, and it has time for the Senate to follow the House! https://t.co/2KxkpaXQ5X
Today I am helping unveil a new Senate Democrats' Special Committee on Climate Change. I am excited to be joining my colleagues today, and I am ready to work with any member of Congress who takes what science is telling us about climate change seriously. https://t.co/ZopPzkY1j4
WATCH: I am joining SenateDems to unveil our new Special Committee on the Climate Crisis. While Mitch McConnell refuses to acknowledge basic science and facts, we are pressing ahead and highlighting the costs of climate change inaction: https://t.co/c6sW8SRVqe
Today we are launching t

In these writings Lee takes controversial positions on issues involving race, civil rights and voting rights. Nominees for lifetime appointments to the federal bench must be forthcoming, regardless of which party nominates them. This is not a partisan issue.
Today, Senate Republicans are holding a hearing for a judicial nominee for whom neither SenKamalaHarris nor I have returned a blue slip. Kenneth Lee repeatedly failed to turn over writings and submitted many only after we identified missing articles.
Neomi Rao has an extreme and troubling record on LGBTQ rights. The SenateGOP must not confirm her to a lifetime appointment on the DC Circuit which will review future rules and regulations intended to protect the rights of LGBTQ Americans. https://t.co/uazDzJ8rEz
The contrast could not be any more clear. Senator MazieHirono argued to pass a resolution calling on TheJusticeDept to protect individuals with pre-existing conditions. Senate Republicans stopped it from happening. https://t.c

All SenateDems opposed Eric Miller's confirmation. He built a career challenging tribal sovereignty &amp; treaty rights. The 9th Circuit is home to almost 75% of Native Nations. How can he fairly hear cases on tribal rights? The SenateGOP chose the hard right over tribal rights.
.SenWhitehouse, MartinHeinrich, and I are sending senatemajldr a letter challenging him to debate climate change with Democrats on the Senate floor. Republicans have held the majority in the Senate for four years and done have nothing on climate. it has time for a change. https://t.co/4rTAMMevRW
We cannot let senatemajldr ignore the fight to #ActOnClimate any longer. We are asking for a debate on the Senate Floor to let the American people hear where both sides stand on the most pressing issue facing our nation. https://t.co/dsrwJxolQw
Republicans have one plan when it comes to climate change: Deny. Deny. Deny. https://t.co/Ud39f1AfLe
Reports say realDonaldTrump plans to set up a fake panel at the WhiteHouse of

...

Clean up the mess in the White House, and let Mueller do his job. We must pass legislation now to protect the Special Counsel investigation from the President. https://t.co/x9qjujwB2Z
50 years ago, Shirley Chisholm—a fellow Brooklynite—became the first African-American woman elected to Congress. I am proud to introduce a Senate resolution, along with RepYvetteClarke and RepBarbaraLee in the House, to honor her achievements and her legacy of public service. https://t.co/FDyfuvy9mF
Just last month, the Trump admin announced its intention to change the definition of 'gender' &amp; if they follow through, it would roll back protections for trans Americans. We must fight back against this. I stand with transgender Americans, especially our youth. You #WontBeErased https://t.co/dYndY394Ks
.SenKamalaHarris &amp; I are introducing a bill to stop the administration from using taxpayer dollars to incite fear &amp; prioritize the well-being of unaccompanied children by placing them with families 

The Republican health care agenda? It would mean cutting coverage for critical care like mental health treatment, substance abuse care for things like opioid addiction, and maternity care.
What we know about President Trump's Health Care agenda: He supports letting employers impose lifetime &amp; annual limits on critical types of health care. That means coverage could be cut off just when you get sick &amp; need it most. But is not that the whole point of health care?
Starting Thursday, you can sign up for health care or renew a plan through the #ACA at https://t.co/NMhRtuxJFb. it has vital that every single New Mexican have access to affordable health care that meets their needs— that is why I am committed to protecting &amp; strengthening the #ACA. https://t.co/45CMrW4uXs
Open enrollment for your 2019 health insurance begins on Thursday, November 1! Make sure you have coverage for 2019 by visiting https://t.co/5DfoxVJJ4N. https://t.co/VlqYO2U9SR
Today on Boston Common, we stood w #P

This is the future Republicans want. https://t.co/AzVk6NVuj1
After blowing a $2 trillion hole in the deficit, the #GOPTaxScam failed to boost wages or create new jobs. Right on time, Republicans in Congress are coming after Americans' Medicare, Medicaid &amp; Social Security to foot the bill. https://t.co/PeuLvOmEUd
After blowing a $2 trillion hole in the deficit to give tax breaks to corporate execs, you can bet the people footing the bill will not be the wealthy &amp; well connected. The GOP will pay for it by cutting your: ✔️Medicare ✔️Medicaid ✔️Social Security https://t.co/9O2aT9Df7x
Here's what is at stake: ending coverage for pre-existing conditions, gutting Medicaid as we know it, and higher premium costs for Americans and their families. https://t.co/MhLb8h0Syz
Now Republicans want to pay down the deficit by cutting Social Security &amp; Medicare! Democrats are never going to let that happen. I have got a better idea: rescind the tax cuts for the wealthy and do not take away b

The Kaiser Family Foundation looked at junk plans in 45 states and found that: 43% did not include coverage for mental health services, 71% did not cover outpatient prescription drugs, and not a single one covered maternity care. — SenCortezMasto https://t.co/1Qt68FoDko
Senate Democrats are committed to fighting for quality, affordable access to physical and mental health care for all Americans. #WorldMentalHealthDay https://t.co/pn7TMJu4MG
The GOP love talking about health care but never mention that they always vote AGAINST efforts to protect families. Case in point: SenatorBaldwin's plan to stop Trump's expansion of “junk plans.” Now those with pre-existing conditions will suffer and millions will face ↑ costs. https://t.co/t3g2ILSmmr
"Hispanic Heritage Month is very important to New Mexico. You cannot tell the story of New Mexico without paying tribute to the contributions of our state's Hispanic ancestors and communities." — SenatorTomUdall https://t.co/dJR5kIvAGZ
Massachusetts ha

KeyboardInterrupt: 